<a href="https://colab.research.google.com/github/Abdul-ur-Rehman-Butt/2022-SE-27_LAB_TASKS_ML/blob/main/lab_12__2022_SE_27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install openpyxl

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore

In [ ]:
data  = pd.read_excel("C:\\Users\\hp\Desktop\\retail_d.xlsx", engine="openpyxl")

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\hp\AppData\Local\Temp\ipykernel_9056\3599991774.py:1: SyntaxWarning: invalid escape sequence '\D'
  data  = pd.read_excel("C:\\Users\\hp\Desktop\\retail_d.xlsx", engine="openpyxl")


In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
# Fill missing numerical values with 0
data.fillna({"CustomerID": data["CustomerID"].median(),  # Use median for ID
           "Description": "Unknown",  # Use 'Unknown' for missing descriptions
           "Quantity": 0,
           "UnitPrice": 0}, inplace=True)


In [ ]:
# Drop rows with missing CustomerID
data_cleaned = data.dropna(subset=["CustomerID"])

In [ ]:
data.describe()

In [ ]:
# Remove outliers using the IQR method
Q1 = data_cleaned[['Quantity', 'UnitPrice']].quantile(0.25)
Q3 = data_cleaned[['Quantity', 'UnitPrice']].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df_filtered = data_cleaned[~((data_cleaned[['Quantity', 'UnitPrice']] < lower_bound) |
                           (data_cleaned[['Quantity', 'UnitPrice']] > upper_bound)).any(axis=1)]

In [ ]:
# Aggregate customer purchasing behavior
customer_data = df_filtered.groupby("CustomerID").agg(
    TotalSpent=("UnitPrice", lambda x: (x * df_filtered.loc[x.index, "Quantity"]).sum()),
    TotalQuantity=("Quantity", "sum"),
    UniqueProducts=("StockCode", "nunique")
).reset_index()

In [ ]:
# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(customer_data.iloc[:, 1:])  # Exclude CustomerID

In [ ]:
# Determine the optimal K using the Elbow method
inertia = []
K_range = range(1, 11)
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(scaled_data)
    inertia.append(kmeans.inertia_)

In [ ]:
# Plot the Elbow Method
plt.figure(figsize=(8, 5))
plt.plot(K_range, inertia, marker='o', linestyle='-')
plt.xlabel("Number of Clusters (K)")
plt.ylabel("Inertia")
plt.title("Elbow Method for Optimal K")
plt.show()

In [ ]:
# Apply K-Means clustering
optimal_k = 4  # Choose based on the Elbow Method plot
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
customer_data["Cluster_KMeans"] = kmeans.fit_predict(scaled_data)

In [ ]:
# Apply DBSCAN clustering
dbscan = DBSCAN(eps=1.2, min_samples=5)  # Adjust eps and min_samples as needed
customer_data["Cluster_DBSCAN"] = dbscan.fit_predict(scaled_data)


In [ ]:
# Compare clustering results
print("K-Means Clustering Results:")
print(customer_data["Cluster_KMeans"].value_counts())

print("\nDBSCAN Clustering Results:")
print(customer_data["Cluster_DBSCAN"].value_counts())

In [ ]:
# Visualize the clustering results
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.scatter(scaled_data[:, 0], scaled_data[:, 1], c=customer_data["Cluster_KMeans"], cmap='viridis', alpha=0.5)
plt.title("K-Means Clustering")

plt.subplot(1, 2, 2)
plt.scatter(scaled_data[:, 0], scaled_data[:, 1], c=customer_data["Cluster_DBSCAN"], cmap='plasma', alpha=0.5)
plt.title("DBSCAN Clustering")

plt.show()
